In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas numpy datasets transformers rouge_score

import pandas as pd
import re
from datasets import Dataset
from sklearn.model_selection import train_test_split

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5d34ee021f9b15b4333231a2bc6182e6c00f2a3c75179cb9408e9820a0403444
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are

In [ ]:
comp_scholar = pd.read_csv("/content/compscholar.csv")
comp_scholar = comp_scholar.dropna(subset=["Document", "Summary"])

In [ ]:
comp_scholar = comp_scholar.sample(n=min(100, len(comp_scholar)), random_state=42).reset_index(drop=True)

In [ ]:
def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

In [ ]:
comp_scholar["Document"] = comp_scholar["Document"].apply(clean_text)
comp_scholar["Summary"] = comp_scholar["Summary"].apply(clean_text)

In [ ]:
data = comp_scholar[["Document", "Summary"]].rename(columns={"Document": "text", "Summary": "summary"})

In [ ]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    model_inputs = tokenizer(
        example["text"], truncation=True, padding="max_length", max_length=512
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["summary"], truncation=True, padding="max_length", max_length=200
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text", "summary"])
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=["text", "summary"])

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=100,
    save_steps=500,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    fp16=False,
    bf16=True,
    save_total_limit=1,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)


trainer.train()

<ipython-input-14-171f71c6e54f>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=30, training_loss=3.2252235412597656, metrics={'train_runtime': 99.358, 'train_samples_per_second': 2.416, 'train_steps_per_second': 0.302, 'total_flos': 185749450260480.0, 'train_loss': 3.2252235412597656, 'epoch': 3.0})

In [ ]:
import torch
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

nltk.download('punkt')

def generate_summary(text, tokenizer, model, max_input_length=512, max_output_length=200):
    inputs = tokenizer(
        text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_input_length
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_output_length,
        num_beams=10,
        length_penalty=1.5,
        no_repeat_ngram_size=3,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

decoded_preds, decoded_refs = [], []

model.eval()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
for i, sample in enumerate(tokenized_val):
    text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    ref = tokenizer.decode(sample["labels"], skip_special_tokens=True)
    pred = generate_summary(text, tokenizer, model)
    decoded_preds.append(pred)
    decoded_refs.append(ref)
    if (i + 1) % 10 == 0:
        print(f"Processed {i+1}/{len(tokenized_val)} samples.")

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

Processed 10/20 samples.
Processed 20/20 samples.


In [ ]:
def compute_rouge(predictions, references):
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        scores["rouge1"].append(score["rouge1"].fmeasure)
        scores["rouge2"].append(score["rouge2"].fmeasure)
        scores["rougeL"].append(score["rougeL"].fmeasure)
    return {key: (sum(scores[key]) / len(scores[key])) * 100 for key in scores}

rouge_results = compute_rouge(decoded_preds, decoded_refs)
print("ROUGE Evaluation Results:", rouge_results)

ROUGE Evaluation Results: {'rouge1': 54.210856353992185, 'rouge2': 28.221067399945166, 'rougeL': 37.96467511631933}


In [ ]:
def compute_bleu(predictions, references):
    smooth_fn = SmoothingFunction().method4
    tokenized_preds = [pred.split() for pred in predictions]
    tokenized_refs = [[ref.split()] for ref in references]
    return corpus_bleu(tokenized_refs, tokenized_preds, smoothing_function=smooth_fn) * 100

bleu_score_value = compute_bleu(decoded_preds, decoded_refs)
print("BLEU Score:", bleu_score_value)

BLEU Score: 18.68971420162931


In [ ]:
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/fine-tuned-model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/fine-tuned-model/tokenizer_config.json',
 '/content/drive/MyDrive/fine-tuned-model/special_tokens_map.json',
 '/content/drive/MyDrive/fine-tuned-model/vocab.json',
 '/content/drive/MyDrive/fine-tuned-model/merges.txt',
 '/content/drive/MyDrive/fine-tuned-model/added_tokens.json',
 '/content/drive/MyDrive/fine-tuned-model/tokenizer.json')